# HAR PhD Thesis - Google Colab Notebook

This notebook provides an interactive way to run the HAR thesis experiments on Google Colab.

**Instructions:**
1. Run the Setup cell first
2. Run chapters sequentially (4 → 5 → 6 → 7)
3. Modify parameters in each cell as needed
4. Use `--help` flag to see available parameters: `!python scripts/ch4_grid_search_om_att.py --help`

**Note:** All scripts are idempotent - they skip already-trained models.

## Setup

Clone repository and install dependencies.

In [ ]:
# Clone repository
!git clone https://github.com/sumeyye-agac/har-phd-thesis.git har-phd-thesis
%cd har-phd-thesis

# Install dependencies (Colab-compatible)
# Note: TensorFlow is already installed in Colab, skip it to avoid version conflicts
# Install only the packages that are compatible with Colab's TensorFlow version
!pip install pandas scikit-learn matplotlib tqdm requests

# Set PYTHONPATH (both sys.path and environment variable for !python commands)
import sys
import os

current_dir = os.getcwd()
os.environ['PYTHONPATH'] = current_dir  # For !python commands
sys.path.insert(0, current_dir)  # For direct Python imports

# Verify setup
import tensorflow as tf
print(f"✅ TensorFlow: {tf.__version__}")
print(f"✅ PYTHONPATH set to: {os.environ.get('PYTHONPATH')}")
print(f"✅ Setup complete!")
print(f"Current directory: {current_dir}")

## Chapter 4: Attention Models

Train baseline OM and run OM-Att grid search.

### 4.1 Train Baseline OM

In [ ]:
# Train baseline OM (Original Model)
!python scripts/ch4_train_om.py

### 4.2 OM-Att Grid Search

**Default (Full Grid Search - 60 models):**
- CH_ATT: 12 models (3 ratios × 4 layers)
- SP_ATT: 12 models (3 kernels × 4 layers)
- CBAM: 36 models (3 ratios × 3 kernels × 4 layers)

**Customize parameters below:**

In [ ]:
# Full grid search (default - all 60 models)
!python scripts/ch4_grid_search_om_att.py

In [ ]:
# Custom: Only CH_ATT with specific parameters
# Modify parameters as needed:
#   ch_att=true|false, sp_att=true|false, cbam=true|false
#   channel_ratios=2,4,8 (comma-separated, no spaces)
#   spatial_kernels=3,5,7 (comma-separated, no spaces)
#   layer_positions=1,2,3,4 (comma-separated, no spaces)

!python scripts/ch4_grid_search_om_att.py ch_att=true sp_att=false cbam=false channel_ratios=2,4 layer_positions=1,2

In [ ]:
# Quick test: Single experiment (1 model) - matches custom_run.sh
!python scripts/ch4_grid_search_om_att.py ch_att=true sp_att=false cbam=false channel_ratios=4 layer_positions=1

In [ ]:
# Get help for available parameters
!python scripts/ch4_grid_search_om_att.py --help

## Chapter 5: Lightweight + Attention

Train baseline LM/MM and run grid searches.

### 5.1 Train Baseline Models

In [ ]:
# Train baseline LM (Lightweight Model)
!python scripts/ch5_train_lm.py

In [ ]:
# Train baseline MM (Mid-size Model)
!python scripts/ch5_train_mm.py

### 5.2 LM-Att Grid Search

**Default: 60 models (12 CH + 12 SP + 36 CBAM)**

In [ ]:
# Full grid search (default)
!python scripts/ch5_grid_search_lm_att.py

In [ ]:
# Custom: Quick test (1 model) - matches custom_run.sh
!python scripts/ch5_grid_search_lm_att.py ch_att=true sp_att=false cbam=false channel_ratios=4 layer_positions=1

### 5.3 MM-Att Grid Search

**Default: 60 models (12 CH + 12 SP + 36 CBAM)**

In [ ]:
# Full grid search (default)
!python scripts/ch5_grid_search_mm_att.py

In [ ]:
# Custom: Quick test (1 model) - matches custom_run.sh
!python scripts/ch5_grid_search_mm_att.py ch_att=true sp_att=false cbam=false channel_ratios=4 layer_positions=1

### 5.4 Select Best Models (Chapter 4 & 5)

In [ ]:
!python scripts/select_best_models.py

## Chapter 6: Knowledge Distillation

Run Knowledge Distillation grid searches.

**Note:** RAB-KD-Att requires OM-Att models from Chapter 4. Ensure Chapter 4 is completed first.

### 6.1 RB-KD Grid Search

**Default: 63 models (7 temperatures × 9 alphas)**

In [ ]:
# Full grid search (default)
!python scripts/ch6_grid_search_rbkd.py

In [ ]:
# Custom: Modify parameters as needed
#   temperatures=1,2,4,6,8,10,15 (comma-separated)
#   alphas=0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9 (comma-separated)

!python scripts/ch6_grid_search_rbkd.py temperatures=1,2,4 alphas=0.1,0.2

In [ ]:
# Quick test: Single experiment - matches custom_run.sh
!python scripts/ch6_grid_search_rbkd.py temperatures=4 alphas=0.5

### 6.2 RB-KD-Att Grid Search

**Default: 3,780 models (63 KD × 60 Attention)**

Combines Knowledge Distillation with Attention mechanisms.

In [ ]:
# Full grid search (default - WARNING: 3,780 models, takes very long!)
# !python scripts/ch6_grid_search_rbkd_att.py

In [ ]:
# Custom: Smaller subset (recommended for testing)
# Only CH_ATT with limited KD parameters - matches custom_run.sh
!python scripts/ch6_grid_search_rbkd_att.py ch_att=true sp_att=false cbam=false temperatures=4 alphas=0.5 channel_ratios=4 layer_positions=1

In [ ]:
# Custom: Quick test (1 model) - matches custom_run.sh
!python scripts/ch6_grid_search_rbkd_att.py ch_att=true sp_att=false cbam=false temperatures=4 alphas=0.5 channel_ratios=4 layer_positions=1

### 6.3 RAB-KD-Att Grid Search

**Default: 3,780 models (63 KD × 60 Attention)**

Uses pre-trained OM-Att models from Chapter 4.

**⚠️ Important:** Ensure Chapter 4 OM-Att models exist before running!

In [ ]:
# Full grid search (default - WARNING: 3,780 models, takes very long!)
# !python scripts/ch6_grid_search_rabkd_att.py

In [ ]:
# Custom: Quick test (1 model) - matches custom_run.sh
!python scripts/ch6_grid_search_rabkd_att.py ch_att=true sp_att=false cbam=false temperatures=4 alphas=0.5 channel_ratios=4 layer_positions=1

### 6.4 Select Best Models (Chapter 6)

In [ ]:
!python scripts/select_best_models.py

## Chapter 7: Model Compression

Apply compression techniques to OM model.

In [ ]:
# Compress OM model (TFLite conversion, quantization, pruning)
!python scripts/ch7_compress_om.py

## Evaluation

Evaluate all trained models and generate results.

### Evaluate Baseline Models

In [ ]:
!python scripts/evaluate_baseline_models.py

### Evaluate All Models

In [ ]:
# Evaluate all trained models (TF & TFLite)
!python scripts/evaluate_all.py

## View Results

View CSV results files.

In [ ]:
import pandas as pd
import os

# List available result files
results_dir = 'results'
if os.path.exists(results_dir):
    csv_files = [f for f in os.listdir(results_dir) if f.endswith('.csv')]
    print("Available result files:")
    for f in csv_files:
        print(f"  - {f}")
else:
    print("Results directory not found.")

In [ ]:
# View a specific CSV file (replace filename as needed)
csv_file = 'results/grid_search_om_att.csv'
if os.path.exists(csv_file):
    df = pd.read_csv(csv_file)
    print(f"\n📊 {csv_file}")
    print(f"Total models: {len(df)}")
    print("\nTop 5 models by validation accuracy:")
    print(df[['model_name', 'val_accuracy', 'test_accuracy']].head())
else:
    print(f"File not found: {csv_file}")

## Download Results

Download CSV files from Colab.

In [ ]:
from google.colab import files
import os

# Download all CSV files
results_dir = 'results'
if os.path.exists(results_dir):
    for filename in os.listdir(results_dir):
        if filename.endswith('.csv'):
            filepath = os.path.join(results_dir, filename)
            files.download(filepath)
            print(f"Downloaded: {filename}")
else:
    print("Results directory not found.")

## Tips & Troubleshooting

**Getting Help:**
- Use `--help` flag: `!python scripts/ch4_grid_search_om_att.py --help`

**Parameter Format:**
- Boolean: `ch_att=true` or `ch_att=false`
- Lists: `channel_ratios=2,4,8` (comma-separated, no spaces)
- Case-insensitive: `CH_ATT=true` works the same as `ch_att=true`

**Idempotent Behavior:**
- Scripts skip already-trained models
- To retrain, delete models from `models_saved/tf/`

**Chapter Dependencies:**
- Chapter 6 RAB-KD-Att requires OM-Att models from Chapter 4
- Run chapters in order: 4 → 5 → 6 → 7

**Full Grid Search Warnings:**
- Chapter 6 full grid searches (3,780 models) take very long
- Use custom parameters to test with smaller subsets first